<a href="https://colab.research.google.com/github/Rafaloga/ECG-Paper-Record-to-Digital-Signal-Conversion-Challenge/blob/main/code/User_Friendly_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Challenge: User Friendly App**
For this section of the challenge, a simple web app has been developed to have a user-friendly interface. The app is a Python script for processing and analyzing electrocardiogram (ECG) images. It allows users to upload an ECG image, perform various image processing operations on it, and then extract, analyze, and visualize ECG signal data.
The development was carried out in a Google Colab notebook to facilitate reproducibility and deployment. Streamlit and Ngrok tools were used in it. Streamlit is an open-source Python library that makes it easy to create web applications for data science and machine learning projects. It allows developers to quickly build interactive and data-driven web applications using familiar Python scripting. Ngrok is a service that provides secure tunneling to localhost. It allows you to expose a web server running on your local machine to the internet, making it accessible from anywhere. Ngrok generates a temporary public URL for your locally hosted web app, making it easy to share your work with others. To run the application, you simply need to execute the cells in the notebook.

## **Packages Installation**

In [1]:
# Install the Streamlit package using pip
!pip install streamlit

# Install the Pyngrok package using pip
!pip install pyngrok

# Authenticate Ngrok using the provided token
!ngrok authtoken 2X7qeeVbU4e8C801aLPlzLnnBQf_5BSSmdFyqybkBKwxubD74

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=fc0f96ba93fc472af18374c499762305c78576a462a128a3c185c598b93cf0e9
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## **Web App development**
 In the following cell, the whole app is created, where users can upload an ECG image, and the code performs several image processing steps, such as correcting orientation, rotation, and smoothing. It then extracts individual ECG signal traces, analyzes them, and presents the results. Users can choose which processed image to view, select a specific ECG signal trace for detailed analysis, and download the original and smoothed ECG data in CSV format for further examination.

In [29]:
%%writefile app.py
import streamlit as st
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import base64

# Determines the orientation of the image based on the vertical histogram
def determine_orientation(image):
    # Calculate a vertical histogram by summing pixel values along columns
    hist = np.sum(image, axis=1)

    # Compare the sum of the first half of the histogram to the second half
    top_half = np.sum(hist[:len(hist)//2])
    bottom_half = np.sum(hist[len(hist)//2:])

    # If the top half has more "weight" than the bottom half, it's upside down
    if top_half > bottom_half:
        return cv2.rotate(image, cv2.ROTATE_180)
    else:
        return image

# Function to rotate the image without losing any content
def rotate_image(image, angle):
    # Get the original image dimensions
    height, width = image.shape[:2]

    # Calculate the bounding rectangle of the non-zero pixels in the image
    rect = cv2.boundingRect(cv2.findNonZero(image))

    # Extract the width and height of this bounding rectangle
    width_rotated = rect[2]
    height_rotated = rect[3]

    # Calculate the center of the new and original image
    center_rotated = (width_rotated // 2, height_rotated // 2)
    center_original = (width // 2, height // 2)

    # Get the rotation matrix for the given angle
    matrix = cv2.getRotationMatrix2D(center_original, angle, 1)

    # Decompose the matrix into cosine and sine values
    cos_val = np.abs(matrix[0, 0])
    sin_val = np.abs(matrix[0, 1])

    # Calculate new dimensions after rotation
    new_width = int((height * sin_val) + (width * cos_val))
    new_height = int((height * cos_val) + (width * sin_val))

    # Adjust the translation part of the rotation matrix
    matrix[0, 2] += (new_width / 2) - center_original[0]
    matrix[1, 2] += (new_height / 2) - center_original[1]

    # Apply the affine transformation
    rotated_image = cv2.warpAffine(image, matrix, (new_width, new_height), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated_image

# Corrects the image orientation based on detected lines
def correct_orientation(img):
    # Detect edges in the image
    edges = cv2.Canny(img, 10, 150, apertureSize=3)

    # Detect lines in the image using the Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi/720, threshold=1000, minLineLength=200, maxLineGap=5)

    angle = 0
    if lines is not None:
        angles = []

        # For each detected line, compute its angle
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angles.append(np.arctan2(y2 - y1, x2 - x1) * 180.0 / np.pi)

        # Use the median angle for rotation
        angle = np.median(angles)

    return rotate_image(img, angle)  # Use the rotate_image function

# Function to extract traces from a binary image
def extract_traces(binary_global):
    contours, _ = cv2.findContours(binary_global, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours in descending order of their area
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

    # Create a list to store the cropped regions of the original image
    cropped_original = []

    # List to store the coordinates of the cropped regions
    crops_coordinates = []

    # Filter contours, crop them, and save them in the list
    for contour in contours:
        # Filter contours based on a minimum area threshold
        # (1000 pixels in this case for obtaining the 2 areas mentioned before)
        if cv2.contourArea(contour) > 1000:
            # Get the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Increase the size of the bounding rectangle by 20% in each dimension
            # beacause without this enlargement, in the next step when the traces
            # are differentiated, some of them would have truncated areas,
            # and therefore, the optimal result would not be achieved.
            increase_percent = 0.20  # 20% increase
            x -= int(w * increase_percent / 2)
            y -= int(h * increase_percent / 2)
            w = int(w * (1 + increase_percent))
            h = int(h * (1 + increase_percent))

            # Ensure that the coordinates are not negative
            x = max(0, x)
            y = max(0, y)

            # Crop the region of interest from the binary image
            cropped_img = binary_global[y:y+h, x:x+w]

            # Append the cropped region and its coordinates to their respective lists
            cropped_original.append(cropped_img)
            crops_coordinates.append((x, y, w, h))

    # Vertical edge detection using Sobel operator as the traces are separated by a vertical line.
    sobelx = cv2.Sobel(cropped_original[0], cv2.CV_64F, 1, 0, ksize=3)

    # Create a copy of the cropped image with the 12 traces
    crop = cropped_original[0].copy()

    # Dilate the edges to enhance and connect them
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(sobelx, kernel, iterations=1)

    # Detect lines using the Hough Line Transform
    lines = cv2.HoughLinesP(crop.astype(np.uint8), 1, np.pi/1000, 10, minLineLength=300, maxLineGap=1)

    # Draw lines on the image
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(crop, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Segment the image based on detected lines
    for i in range(len(lines) - 1):
        lead = crop[:, lines[i][0][0]:lines[i+1][0][0]]

    # Find contours that correspond to the traces
    contours, _ = cv2.findContours(crop, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get bounding boxes and their coordinates (x, y, width, height) and filter by area
    bounding_boxes = [(cv2.boundingRect(contour), contour) for contour in contours if cv2.contourArea(contour) > 2000]

    # Get the (x, y, width, height) coordinates of the bounding boxes
    coordinates = [(box[0], box[1], box[2], box[3]) for box, _ in bounding_boxes]

    # Sort the coordinates, as they were placed un a 3x4 layout
    # Sort the coordinates by the y-coordinate
    coordinates.sort(key=lambda coord: coord[1])

    # Divide the coordinates into three parts
    num_parts = 3
    part_size = len(coordinates) // num_parts

    parts = [coordinates[i:i+part_size] for i in range(0, len(coordinates), part_size)]

    # Sort each part by the x-coordinate
    for i, part in enumerate(parts):
        parts[i] = sorted(part, key=lambda coord: coord[0])

    sorted_coordinates = [coord for part in parts for coord in part]

    # Create a list to store cropped images
    cropped_images = []
    cropped_coordinates = []  # Create a list to store coordinates

    # Filter contours and crop following the order, and adjust size by 20% in height only.
    # The height adjustment is made to ensure the whole signal is captured in the rectangle
    for index, (x, y, w, h) in enumerate(sorted_coordinates):
        # Calculate a 20% increase in height
        increase_percent = 0.20  # 20% increase
        h_increase = int(h * increase_percent / 2)

        # Adjust coordinates to increase size by 20% in height only
        y -= h_increase
        h += h_increase * 2

        # Make sure coordinates are not negative
        y = max(0, y)

        # Crop the image using the adjusted coordinates
        final = crop[y:y+h, x:x+w]
        cropped_images.append(final)  # Add the cropped image to the list

        # Store the coordinates in the cropped_coordinates list
        cropped_coordinates.append((x, y, w, h))

    # Add the 13th trace to the list
    cropped_images.append(cropped_original[1])
    cropped_coordinates.append(crops_coordinates[1])

    return cropped_images, cropped_coordinates

# Get contours from traces
def get_contours_from_traces(cropped_images, cropped_coordinates):
    original_contours = []
    largest_contours = []
    absolute_contours = []  # List to store contours in terms of the entire image


    # Loop through all crops in cropped_images
    for index, image in enumerate(cropped_images):
        if index >= len(cropped_coordinates):
            continue
        # Find contours in the current image
        contours = measure.find_contours(image, 0.1)

        # Find the largest contour shape
        contours_shapes = sorted([x.shape for x in contours])[::-1][0:1]

        # Store the largest contour in this variable
        largest_contour = None

        # Store the original contour
        original_contour = None

        # Find the largest contour in this image
        for contour in contours:
            if contour.shape in contours_shapes:
                # Resize the contour to an arbitrary value (it can vary depending on the requirements of the application)
                # This is made to ensure all the signals have the same size, so they can be stored and compared easily.
                resized_contour = resize(contour, (1023, 2))
                # Store the largest contour
                largest_contour = resized_contour
                original_contour = contour

        # Add the largest and original contour to the lists
        largest_contours.append(largest_contour)
        original_contours.append(original_contour)

        # Now, adjust the original_contour's coordinates to be relative to the original image
        # This way all the signals can be plotted again together
        x_offset, y_offset, _, _ = cropped_coordinates[index]
        absolute_contour = np.array([[y + y_offset, x + x_offset] for y, x in original_contour])
        absolute_contours.append(absolute_contour)

    return original_contours, largest_contours, absolute_contours

# Show a contour
def show_contour(contour):
    fig, ax = plt.subplots(figsize=(5, 2))

    ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='black')
    ax.axis('image')
    ax.invert_yaxis()
    ax.axis('off')

    plt.tight_layout()

    return fig

# Smooth and normalize contours
def smooth_and_normalize_contours(absolute_contours, window_size=10):
    original_data_list = []
    smoothed_data_list = []
    # Number of signals
    num_signals = len(absolute_contours)

    # Loop through the signals
    for i in range(num_signals):
        # Get the original data for the current signal
        original_data = absolute_contours[i]

        # Apply moving average smoothing to the current signal
        df = pd.DataFrame(original_data, columns=['Y', 'X'])
        central_line = df.groupby('X')['Y'].mean().reset_index()
        central_line['Y_smooth'] = central_line['Y'].rolling(window=window_size, center=True).mean()

        # Add the smoothed DataFrame to the list
        smoothed_data_list.append(central_line)

        # Add the original data to the list
        original_data_list.append(original_data)


    return original_data_list, smoothed_data_list

# Save data to CSV
def save_to_csv(original_data_list, smoothed_data_list):
    # Create DataFrames from the original data and add a 'Signal' column so they can be found in the csv file
    original_data_list_df = [pd.DataFrame(data, columns=['Y', 'X']) for data in original_data_list]
    for i, df in enumerate(original_data_list_df):
        df['Signal'] = i + 1  # Assign a unique signal value to each data set

    # Create DataFrames from the smoothed data and add a 'Signal' column so they can be found in the csv file
    smoothed_data_list_df = [pd.DataFrame(data, columns=['X', 'Y_smooth', 'Y']) for data in smoothed_data_list]
    for i, df in enumerate(smoothed_data_list_df):
        df['Signal'] = i + 1  # Assign a unique signal value to each smoothed data set

    # Concatenate all the original data DataFrames into one
    original_data = pd.concat(original_data_list_df, ignore_index=True)

    # Concatenate all the smoothed data DataFrames into one
    smoothed_data = pd.concat(smoothed_data_list_df, ignore_index=True)

    # Initialize the MinMaxScaler to normalize the 'X', 'X_smooth', and 'Y' columns
    scaler = MinMaxScaler()

    # Normalize the 'X', 'X_smooth', and 'Y' columns in the smoothed data
    smoothed_data[['X', 'Y_smooth', 'Y']] = scaler.fit_transform(smoothed_data[['X', 'Y_smooth', 'Y']])


    csv_filename_original = 'original_data.csv'
    original_data.to_csv(csv_filename_original, index=False)

    csv_filename = 'normalized_data.csv'
    smoothed_data.to_csv(csv_filename, index=False)

    return csv_filename_original, csv_filename

# Get a download link for a CSV file
def get_csv_download_link(csv_filename, link_name):
    with open(csv_filename, 'rb') as f:
        csv_data = f.read()
    b64 = base64.b64encode(csv_data).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{csv_filename}">{link_name}</a>'
    return href

# Display graphs
def display_graphs(original_data_list, smoothed_data_list, selected_signal_index):
    col1, col2 = st.columns(2)

    # Create the original data graph
    fig1, ax1 = plt.subplots(figsize=(6, 3))
    ax1.plot(original_data_list[selected_signal_index][:, 1], original_data_list[selected_signal_index][:, 0], linewidth=1, color='black')
    ax1.set_title(f'Signal {selected_signal_index + 1} - Original Data')
    ax1.invert_yaxis()

    # Show the original data graph in the first column
    with col1:
        st.pyplot(fig1)

    # Create the smoothed data graph
    fig2, ax2 = plt.subplots(figsize=(6, 3))
    ax2.plot(smoothed_data_list[selected_signal_index]['X'], smoothed_data_list[selected_signal_index]['Y_smooth'], linewidth=1, color='black', linestyle='solid')
    ax2.set_title(f'Signal {selected_signal_index + 1} - Smoothed Data')
    ax2.invert_yaxis()

    # Show the smoothed data graph in the second column
    with col2:
        st.pyplot(fig2)

def show_all_images(csv_filename_original):
    original_data = pd.read_csv(csv_filename_original)

    # Determine the number of unique signals (assuming each signal has a unique identifier)
    num_signals = original_data['Signal'].nunique()

    # Create a new figure with appropriate size
    fig, ax = plt.subplots(figsize=(10, 6))

    # Loop through each unique signal
    for signal_id in range(1, num_signals + 1):
        # Filter the original data for the current signal
        original_signal_data = original_data[original_data['Signal'] == signal_id]

        # Plot the data of the current signal
        ax.plot(original_signal_data['X'], original_signal_data['Y'], label=f'Signal {signal_id}')

    # Configure the plot
    ax.set_title('All Signals - Original Data')
    ax.invert_yaxis()  # Invert the Y-axis if necessary
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Add a legend to the plot

    return fig

def main():
    st.title("Conversion of Paper ECG to Digital Signal")

    # Load the image
    uploaded_file = st.file_uploader("Upload an ECG record image", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = cv2.imdecode(np.fromstring(uploaded_file.read(), np.uint8), cv2.IMREAD_UNCHANGED)

        # Apply transformations to the image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_corrected = determine_orientation(gray)
        img_rotated = correct_orientation(img_corrected)
        blurred_image = cv2.GaussianBlur(img_rotated, (5, 5), 0.7)
        _, binary_global = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        traces, traces_coordinates = extract_traces(binary_global)
        original_contours, largest_contours, absolute_contours = get_contours_from_traces(traces, traces_coordinates)

        # Create a dropdown to select which image to display
        option = st.selectbox(
            'Select an image to display',
            ('Original', 'Grayscale', 'Corrected', 'Rotated', 'Smoothed', 'Binary')
        )

        # Display the selected image
        if option == 'Original':
            st.image(image, channels="BGR", caption="Original Image", use_column_width=True)
        elif option == 'Grayscale':
            st.image(gray, caption="Grayscale Image", use_column_width=True)
        elif option == 'Corrected':
            st.image(img_corrected, caption="Corrected Image", use_column_width=True)
        elif option == 'Rotated':
            st.image(img_rotated, caption="Rotated Image", use_column_width=True)
        elif option == 'Smoothed':
            st.image(blurred_image, caption="Smoothed Image", use_column_width=True)
        elif option == 'Binary':
            st.image(binary_global, caption="Binary Image", use_column_width=True)

        # Create a single dropdown to select the signal
        signal_option = st.selectbox('Select a signal to display', ["Signal " + str(i+1) for i in range(len(traces))])
        selected_signal_index = int(signal_option.split(" ")[-1]) - 1

        # Create two columns to display the trace and contour of the selected signal
        col1, col2 = st.columns(2)

        with col1:
            st.image(traces[selected_signal_index], caption="Trace of " + signal_option, use_column_width=True)
        with col2:
            st.pyplot(show_contour(absolute_contours[selected_signal_index]))

        # Smooth and normalize
        original_data_list, smoothed_data_list = smooth_and_normalize_contours(absolute_contours)

        # Dropdown to select signal to graph
        selected_graph_signal = st.selectbox('Select a signal to graph', ["Signal " + str(i+1) for i in range(len(original_data_list))])
        selected_graph_signal_index = int(selected_graph_signal.split(" ")[-1]) - 1

        # Call the function to display graphs
        display_graphs(original_data_list, smoothed_data_list, selected_graph_signal_index)

        # Save to CSV files
        csv_filename_original, csv_filename = save_to_csv(original_data_list, smoothed_data_list)

        # Create a dropdown to select which final image to display
        option = st.selectbox(
            'Select an the data to display',
            ('Original', 'Normalized')
        )

        # Display the selected image
        if option == 'Original':
            fig = show_all_images(csv_filename_original)
            st.pyplot(fig)
        elif option == 'Normalized':
            fig = show_all_images(csv_filename)
            st.pyplot(fig)



        # Provide download links for CSV files
        st.write('Download the data:')
        if st.button('Download Original Data'):
            st.markdown(get_csv_download_link(csv_filename_original, 'Download Original Data'), unsafe_allow_html=True)

        if st.button('Download Normalized Data'):
            st.markdown(get_csv_download_link(csv_filename, 'Download Normalized Data'), unsafe_allow_html=True)



if __name__ == "__main__":
    main()


Overwriting app.py


## **Streamlit App Deployment with Ngrok**
This cell runs the Streamlit application (app.py) in the background using nohup. It then employs Pyngrok to create a public tunnel, assigning a public URL to the locally running Streamlit app on port 8501. The printed public_url represents the publicly accessible URL for the Streamlit application, allowing users to access it from a web browser or any remote location via the Internet. This combination of Streamlit and Ngrok enables the deployment of the Streamlit app with a public URL for remote access and sharing.

In [33]:
!nohup streamlit run app.py &

from pyngrok import ngrok

# Configura el túnel
public_url = ngrok.connect(8501)
print(public_url)

nohup: appending output to 'nohup.out'


NgrokTunnel: "https://e14b-35-245-66-48.ngrok-free.app" -> "http://localhost:8501"


In [34]:
# Kill all Streamlit processes
!pkill -f "streamlit"

# Kill all Ngrok processes
!pkill -f "ngrok"
